## 实验：Dropout 算法

除了权重衰减以外，深度学习模型常常使用 **丢弃法（dropout）** 来应对过拟合问题。丢弃法有一些不同的变体。本实验中提到的丢弃法特指：**倒置丢弃法（inverted dropout）**。

### 实验概要

#### Dropout 算法

多层感知机在单层神经网络的基础上引入了一到多个隐藏层（hidden layer）。隐藏层位于输入层和输出层之间。

![Alt text](./img/mlp_21.svg)

设：输入个数为 `4`，隐藏单元个数为 `5`，且隐藏单元 $h_{i}(i=1, \ldots, 5)$ 的计算表达式为：

$$h_{i}=\phi\left(x_{1} w_{1 i}+x_{2} w_{2 i}+x_{3} w_{3 i}+x_{4} w_{4 i}+b_{i}\right)$$

这里 $\phi$ 是激活函数，$x_{1}, \ldots, x_{4}$ 是输入，隐藏单元 $i$ 的权重参数为 $w_{1 i}, \ldots, w_{4 i}$，偏置参数为 $b_i$。当对该隐藏层使用丢弃法时，该层的隐藏单元将有一定概率被丢弃掉。设丢弃概率为 $p$，那么， $h_i$ 有 $p$ 的概率会被清零，有 $(1 − p)$ 的概率会除以 $(1 − p)$ 做拉伸。丢弃概率是丢弃法的超参数。具体来说，设随机变量 $\xi_{i}$ 为 0 和 1 的概率分别为 $p$ 和 $(1 − p)$，使用丢弃法时我们计算新的隐藏单元 $h_{i}^{\prime}$：

$$
h_{i}^{\prime}=\frac{\xi_{i}}{1-p} h_{i}
$$

由于 $E\left(\xi_{i}\right)=1-p$，因此：

$$
E\left(h_{i}^{\prime}\right)=\frac{E\left(\xi_{i}\right)}{1-p} h_{i}=h_{i}
$$

即**丢弃法不改变其输入的期望值**。如对上图的隐藏层使用丢弃法，一种可能的结果如下图所示，其中 $h_{2}$ 和 $h_{5}$ 被清零。这时输出值的计算不再依赖 $h_{2}$ 和 $h_{5}$，在反向传播时，与这两个隐藏单元相关的权重的梯度均为 `0`。由于在训练中隐藏层神经元的丢弃是随机的，即 $h_{1}, \ldots, h_{5}$ 都有可能被清零，输出层的计算无法过度依赖 $h_{1}, \ldots, h_{5}$ 中的任一个，从而在训练模型时起到正则化的作用，并可以用来应对过拟合。在测试模型时，我们为了拿到更加确定性的结果，一般不使用丢弃法。

![Alt text](./img/dropout_21.svg)

### 实验目标

在本实验中，我们将根据丢弃法的定义，通过 Python 实现它，之后我们会通过 Tensorflow 框架作相同的测试。

### 1. 导入库

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras, nn, losses
from tensorflow.keras.layers import Dropout, Flatten, Dense

### 2. 定义 Dropout 函数

我们将通过定义一个 `dropout()` 函数，以 `drop_prob` 的概率丢弃 X 中的元素。

In [2]:
def dropout(X, drop_prob):
    assert 0 <= drop_prob <= 1
    keep_prob = 1 - drop_prob
    
    # 这种情况下把全部元素都丢弃
    if keep_prob == 0:
        return tf.zeros_like(X)
    
    #初始mask为一个bool型数组，故需要强制类型转换
    mask = tf.random.uniform(shape=X.shape, minval=0, maxval=1) < keep_prob
    return tf.cast(mask, dtype=tf.float32) * tf.cast(X, dtype=tf.float32) / keep_prob

我们运行几个例子来测试一下 `dropout()` 函数。其中丢弃概率分别为 `0`、`0.5` 和 `1`。

In [3]:
X = tf.reshape(tf.range(0, 16), shape=(2, 8))
dropout(X, 0)

<tf.Tensor: shape=(2, 8), dtype=float32, numpy=
array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11., 12., 13., 14., 15.]], dtype=float32)>

上面全部隐藏层单元被保留。

In [4]:
dropout(X, 0.5)

<tf.Tensor: shape=(2, 8), dtype=float32, numpy=
array([[ 0.,  0.,  0.,  6.,  8., 10.,  0.,  0.],
       [ 0.,  0.,  0.,  0., 24., 26., 28.,  0.]], dtype=float32)>

上面部分隐藏层单元被保留，剩下的一部分在原有值的基础上除以 $(1 - p)$，我们设置了 `p = 0.5`，即乘以 2.

In [5]:
dropout(X, 1.0)

<tf.Tensor: shape=(2, 8), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>

上面全部隐藏层单元被清零。

### 3. 定义模型参数

下面，我们将定义一个包含两个隐藏层的多层感知机，其中两个隐藏层的输出个数都是 `256`，然后将其代入到 MNIST 数据集中。

In [6]:
num_inputs, num_outputs, num_hiddens1, num_hiddens2 = 784, 10, 256, 256

W1 = tf.Variable(tf.random.normal(stddev=0.01, shape=(num_inputs, num_hiddens1)))
b1 = tf.Variable(tf.zeros(num_hiddens1))
W2 = tf.Variable(tf.random.normal(stddev=0.1, shape=(num_hiddens1, num_hiddens2)))
b2 = tf.Variable(tf.zeros(num_hiddens2))
W3 = tf.Variable(tf.random.truncated_normal(stddev=0.01, shape=(num_hiddens2, num_outputs)))
b3 = tf.Variable(tf.zeros(num_outputs))

params = [W1, b1, W2, b2, W3, b3]

### 4. 定义模型

下面定义的模型将全连接层和激活函数 `ReLU` 串起来，并对每个激活函数的输出使用丢弃法。我们可以分别设置各个层的丢弃概率。通常的建议是把靠近输入层的丢弃概率设得小一点。在这个实验中，我们把第一个隐藏层的丢弃概率设为 `0.2`，把第二个隐藏层的丢弃概率设为 `0.5`。我们可以通过参数 `is_training` 函数来判断运行模式为训练还是测试，并只需在训练模式下使用丢弃法。

In [7]:
drop_prob1, drop_prob2 = 0.2, 0.5

def net(X, is_training=False):
    X = tf.reshape(X, shape=(-1,num_inputs))
    H1 = tf.nn.relu(tf.matmul(X, W1) + b1)
    
    # 只在训练模型时使用丢弃法
    if is_training:
        # 在第一层全连接后添加丢弃层
        H1 = dropout(H1, drop_prob1)  
    H2 = nn.relu(tf.matmul(H1, W2) + b2)
    
    if is_training:
        # 在第二层全连接后添加丢弃层
        H2 = dropout(H2, drop_prob2)  
    
    return tf.math.softmax(tf.matmul(H2, W3) + b3)

### 5. 训练和测试模型

#### 5.1 定义准确率函数

In [8]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for _, (X, y) in enumerate(data_iter):
        y = tf.cast(y,dtype=tf.int64)
        acc_sum += np.sum(tf.cast(tf.argmax(net(X), axis=1), dtype=tf.int64) == y)
        n += y.shape[0]
    return acc_sum / n

#### 5.2 定义模型训练函数

In [9]:
def train_mlp(net, train_iter, test_iter, loss, num_epochs, batch_size,
              params=None, lr=None, trainer=None):
    global sample_grads
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X, y in train_iter:
            with tf.GradientTape() as tape:
                y_hat = net(X, is_training=True)
                l = loss(y_hat, tf.one_hot(y, depth=10, axis=-1, dtype=tf.float32))

            grads = tape.gradient(l, params)
            if trainer is None: # 如果没有传入优化器

                sample_grads = grads
                params[0].assign_sub(grads[0] * lr)
                params[1].assign_sub(grads[1] * lr)
            else:
                trainer.apply_gradients(zip(grads, params)) 

            y = tf.cast(y, dtype=tf.float32)
            train_l_sum += l.numpy()
            train_acc_sum += tf.reduce_sum(tf.cast(tf.argmax(y_hat, axis=1) 
                                == tf.cast(y, dtype=tf.int64), dtype=tf.int64)).numpy()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

$\uparrow$ 由于 Notebook 输出格式限制，其中可能有代码显示不全：

```python
train_acc_sum += tf.reduce_sum(tf.cast(tf.argmax(y_hat, axis=1) == tf.cast(y, dtype=tf.int64), dtype=tf.int64)).numpy()
```

#### 5.3 加载数据

MNIST 是 TensorFlow 的内置数据集，直接从框架中导入即可。

```keras.datasets.mnist.load_data()```

但由于需要从互联网上下载，连接可能不稳定，我们直接下载到本地，使用下面的代码进行加载。

与之前实验一样，训练集与测试据依然是 60000 张 / 10000 张 `28*28` 像素图片。

In [10]:
import os
base_path = os.environ.get("BATH_PATH",'./data/')
data_path = os.path.join(base_path + "lab21/")
result_path = "result/"
os.makedirs(result_path, exist_ok=True)

path = data_path+'mnist.npz'

f = np.load(path)
x_train, y_train = f['x_train'], f['y_train']
x_test, y_test = f['x_test'], f['y_test']
f.close()

### 6. 训练模型

In [11]:
loss = tf.losses.CategoricalCrossentropy()
num_epochs, lr, batch_size = 5, 0.5, 256

# 在进行矩阵相乘时需要 float 型，故强制类型转换为 float 型
x_train = tf.cast(x_train, tf.float32) / 255 

# 在进行矩阵相乘时需要 float 型，故强制类型转换为 float 型
x_test = tf.cast(x_test,tf.float32) / 255 
train_iter = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
test_iter = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
train_mlp(net, train_iter, test_iter, loss, num_epochs, batch_size,
              params, lr)

epoch 1, loss 0.0387, train acc 0.582, test acc 0.758
epoch 2, loss 0.0242, train acc 0.727, test acc 0.840
epoch 3, loss 0.0198, train acc 0.781, test acc 0.865
epoch 4, loss 0.0176, train acc 0.802, test acc 0.877
epoch 5, loss 0.0162, train acc 0.816, test acc 0.885


从上面的训练结果看到训练集的准确率 `train acc`，与测试集的准确率 `test acc`，基本匹配，并没有出现前面的实验中偏离值很严重的情况。

### 7. 使用 TensorFlow 框架实现训练

在 Tensorflow2.0 中，我们只需要在全连接层后添加 Dropout 层并指定丢弃概率。在训练模型时，Dropout 层将以指定的丢弃概率随机丢弃上一层的输出元素；在测试模型时（即 `model.eval()` 后），Dropout 层并不发挥作用。

In [12]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(256,activation='relu'),
    Dropout(0.2),
    keras.layers.Dense(256,activation='relu'),
    Dropout(0.5),
    keras.layers.Dense(10,activation=tf.nn.softmax)
])

下面训练并测试模型。

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train,y_train,epochs=5,
          batch_size=256,
          validation_data=(x_test, y_test),
          validation_freq=1)

Epoch 1/5
235/235 [==============================] - 6s 21ms/step - loss: 0.4341 - accuracy: 0.8709 - val_loss: 0.1526 - val_accuracy: 0.9522
Epoch 2/5
235/235 [==============================] - 5s 19ms/step - loss: 0.1741 - accuracy: 0.9488 - val_loss: 0.1073 - val_accuracy: 0.9653
Epoch 3/5
235/235 [==============================] - 5s 20ms/step - loss: 0.1234 - accuracy: 0.9632 - val_loss: 0.0867 - val_accuracy: 0.9740
Epoch 4/5
235/235 [==============================] - 4s 18ms/step - loss: 0.1003 - accuracy: 0.9688 - val_loss: 0.0771 - val_accuracy: 0.9764
Epoch 5/5
235/235 [==============================] - 4s 19ms/step - loss: 0.0831 - accuracy: 0.9747 - val_loss: 0.0734 - val_accuracy: 0.9775


同样，在 Tensorflow 框架中实现 Dropout，训练集的准确率 `accuracy` 与测试集的准曲率 `val_accuracy` 依然匹配，也没有出现前面的实验中偏离值很严重的情况。

### 实验小结

在本实验中，你根据丢弃法的定义，通过 Python 实现了 Dropout 运算，之后又通过 Tensorflow 框架作相同的测试。